In [ ]:
import torch
from transformers import (AutoTokenizer, AutoConfig,
AutoModelForSequenceClassification)
import datasets

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)